In [1]:
from vocabulary_grouping_tools.ingestion.api_ingest import get_english_translation
import pandas as pd
import yaml

In [2]:
from vocabulary_grouping_tools.grouping.grouping_tools import make_all_subseqs

In [3]:
secrets = yaml.safe_load(open('../metadata/secrets.yml', 'r'))

In [6]:
vocab = pd.read_csv('../input_files_from_output/vocab_active_korean_1_2_ttmik_1_2_refined_with_hanja.csv')

In [7]:
vocab

,Unnamed: 0,index,hangul_word,english_phrase,does_end_in_hada,hanja_word,hanja_exists,hanja_word_with_hada,num_hangul_words
0,3811,이,이,this,False,乁,True,乁,384
1,3812,이,이,this,False,二,True,二,384
2,3813,이,이,this,False,以,True,以,384
3,3814,이,이,this,False,伊,True,伊,384
4,3815,이,이,this,False,伿,True,伿,384
...,...,...,...,...,...,...,...,...,...
6103,9,갈아타다,갈아타다,to transfer,False,NaN,False,NaN,0
6104,4,가르치다,가르치다,to teach,False,NaN,False,NaN,0
6105,3,가다,가다,to go,False,NaN,False,NaN,0
6106,2,가게,가게,store / shop,False,NaN,False,NaN,0


In [8]:
hangul_words = vocab['hangul_word']

In [9]:
max([len(_word) for _word in hangul_words])

5

In [10]:
def cutoff_ending(hangul_word):
    if hangul_word.endswith('하다'):
        return hangul_word[:-2]
    elif hangul_word.endswith('다'):
        return hangul_word[:-1]
    else:
        return hangul_word

In [11]:
subseq_set = set()
for hangul_wrd in list(hangul_words):
    subseq_set.update(make_all_subseqs(cutoff_ending(hangul_wrd)))

In [16]:
make_all_subseqs('대한민국')

['대', '대한', '대한민', '대한민국', '한', '한민', '한민국', '민', '민국', '국']

In [17]:
trans = get_english_translation('인도', secrets['client_id'], secrets['client_secret'])

In [18]:
trans

'India'

In [ ]:
translation_for_subseq = {}
for subseq in subseq_set:
    print('Now getting translation for ' + subseq)
    english_trans = get_english_translation(subseq, secrets['client_id'], secrets['client_secret'])
    print(f'Translation is {english_trans}')
    print()
    translation_for_subseq[subseq] = english_trans

In [30]:
len(translation_for_subseq)

1039

In [31]:
df_of_seq_translation = pd.DataFrame(translation_for_subseq.items()).rename(columns={
    0: 'hangul_syllable',
    1: 'english_translation'
})

In [32]:

df_of_seq_translation.to_csv('../metadata/hangul_subsequence_translation.csv')